<a href="https://colab.research.google.com/github/shruti1721/CodSoft--Task2/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
pip install tensorflow

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
pip install azure-storage-blob

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
dbutils.library.restartPython()

In [ ]:
# %pip install --upgrade protobuf

import cv2
import tensorflow as tf
# import imghdr
import numpy as np
from matplotlib import pyplot as plt

/databricks/python/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
dbutils.fs.ls('/mnt/projectteam5/')

[FileInfo(path='dbfs:/mnt/projectteam5/img_dir/', name='img_dir/', size=0, modificationTime=1707896574000)]

In [ ]:
# import tensorflow as tf

# # Define the directory path in the Databricks file system
# data_directory = '/dbfs/mnt/projectteam5/img_dir/'

# # Define image data generator
# image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# # Load the image dataset
# data = image_data_generator.flow_from_directory(
#     data_directory,
#     target_size=(256, 256),
#     batch_size=32,
#     class_mode='binary',
#     shuffle=True,
#     seed=123
# )


Found 3026 images belonging to 2 classes.


In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load and split image data from directory into training and validation sets
train_data = datagen.flow_from_directory(
    directory='/dbfs/mnt/projectteam5/img_dir/',
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset='training')  # This indicates it's for training data

val_data = datagen.flow_from_directory(
    directory='/dbfs/mnt/projectteam5/img_dir/',
    target_size=(256,256),
    batch_size=32,
    class_mode='binary',
    subset='validation')  # This indicates it's for validation data

Found 2422 images belonging to 2 classes.
Found 604 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense , Flatten , Dropout

In [ ]:
model= Sequential()

In [ ]:
model.add(Conv2D(16,(3,3),1 , activation='relu',padding='same', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32,(3,3),1,padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16,(3,3),1,padding='same',activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile('adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics='accuracy')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 32, 32, 16)        0

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback= tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
tensorboard_callback

In [ ]:
hist= model.fit(train_data, epochs=5,validation_data=val_data , callbacks=[tensorboard_callback])

Epoch 1/5
60/76 [======================>.......] - ETA: 13:56 - loss: 0.7262 - accuracy: 0.5586

In [ ]:
# Test

In [ ]:
img = cv2.imread('/content/drive/MyDrive/proj/Colgate/Bard_Generated_Image (1)_0_2442.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize= tf.image.resize(img,(256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat=model.predict(np.expand_dims(resize/255,0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
  print('Predicted Image is Fake ')
else:
  print('Predicted Image is Original')


In [ ]:
img = cv2.imread('/content/drive/MyDrive/Dataset/Fake Logos/Ariel')
plt.imshow(img)
resize= tf.image.resize(img,(256,256))
plt.imshow(resize.numpy().astype(int))
yhat=model.predict(np.expand_dims(resize/255,0))
yhat

In [ ]:
if yhat > 0.5:
  print('Predicted Image is Fake ')
else:
  print('Predicted Image is Original')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()